In [1]:
import BayesianOptimisation.boVoronoiDirected as boVoronoiDirected
import BayesianOptimisation.expGrid as expGrid
import BayesianOptimisation.expVoronoi as expVoronoi
import BayesianOptimisation.ANNVoronoiDirected as ANNVoronoiDirected
import BayesianOptimisation.experiment_setup as experiment
from core.DataStructure import *

from importlib import reload
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import copy

In [2]:
from scipy.optimize import minimize, Bounds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Reshape
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

In [3]:
np.random.seed(42)
# np.random.seed(43)

In [4]:
reload(ANNVoronoiDirected)

<module 'BayesianOptimisation.ANNVoronoiDirected' from '/home/elim/Documents/Cambridge-Master/Research/code/BayesianOptimisation/ANNVoronoiDirected.py'>

### Generating Data

In [5]:
scene = "./input/random-32-32-10/scen-even/random-32-32-10-even-" + str(0) + ".scen"
exp = experiment.Experiment(scene=scene)

In [6]:
exp.start_nodes

In [7]:
exp.end_nodes

In [8]:
ANNVoronoiDirected.ann_voronoi_directed(exp)

Set Hyper Parameters, solving for 20 agents
total number of distance 514
Number of trainable probabilities 388
Length cutoff threshold 2

 Generating Data, Sample  0
m 0.5012295032817167 c 0.5942596319029034
penality 1.0
global cost 24.803854356726134

 Generating Data, Sample  1
m 0.4563775124072067 c 0.49861493233854415
penality 1.0
global cost 24.803257937531857

 Generating Data, Sample  2
m 0.6060517956653269 c 0.5120303441946381
penality 1.0
global cost 24.80398122831676

 Generating Data, Sample  3
m 0.571303236160464 c 0.38275287785166456
penality 1.0
global cost 24.805095660786687

 Generating Data, Sample  4
m 0.2413209596491681 c 0.4082430795230515
penality 1.0
global cost 24.803656144955433

 Generating Data, Sample  5
m 0.045020362171176886 c 0.36318369382429866
penality 1.0
global cost 24.804528536873118

 Generating Data, Sample  6
m 0.7958383739153316 c 0.6192299076405358
penality 304.3117208417769
global cost 7280.220650492399

 Generating Data, Sample  7
m -0.93511310

KeyboardInterrupt: 

In [ ]:
# pd.DataFrame(X).to_csv("./data/X-1000.csv", index=None)
# pd.DataFrame(y).to_csv("./data/y-1000.csv", index=None)

### Data Processing

In [ ]:
X_read = pd.read_csv("./data/10000-samples/x_path.csv")
X = X_read.to_numpy()
X = X.reshape((-1,989))
X.shape

In [ ]:
Y_read = pd.read_csv("./data/10000-samples/y_path.csv")
y = Y_read.to_numpy()
y.shape

In [ ]:
x_train_temp, x_test, y_train_temp, y_test = train_test_split(
    X, y, test_size=0.1, random_state=4
)
x_train, x_val, y_train, y_val = train_test_split(
    x_train_temp, y_train_temp, test_size=0.11, random_state=4
)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train_norm = scaler.fit_transform(x_train)
y_train_norm = scaler.fit_transform(y_train)
x_val_norm = scaler.fit_transform(x_val)
y_val_norm = scaler.fit_transform(y_val)
x_test_norm = scaler.fit_transform(x_test)
y_test_norm = scaler.fit_transform(y_test)

# inverse = scaler.inverse_transform(normalized)

In [ ]:
x_train_norm = x_train_norm.reshape((-1,1,989))
x_test_norm = x_test_norm.reshape((-1,1,989))
x_val_norm = x_val_norm.reshape((-1,1,989))

### ANN

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
model = keras.models.Sequential([
    Dense(units=989, activation='relu', kernel_initializer='normal', input_shape=(1,989)),
    Dropout(0.2),
    Dense(units=512, activation='relu',),
    Dropout(0.2),
    Dense(units=128, activation='relu',),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid'),
])

opt = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(
    loss="mean_squared_error", 
    optimizer=opt,
    metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train_norm, 
    y_train_norm,
    batch_size = 256,
    epochs=100,
    validation_data=(x_val_norm, y_val_norm))

In [ ]:
test = model.predict(x_test_norm[0])
scaler.inverse_transform(test)

In [ ]:
model.save('./data/10000-samples/model')

### Running Optimiser

In [ ]:
model = keras.models.load_model('./data/10000-samples/model')

In [ ]:
def run_nn(arr):
    arr = np.round(arr, 2)
    c = model.predict(arr.reshape(1,989))
    cost = scaler.inverse_transform(c)
    print("cost", cost)
    return cost

In [ ]:
res = minimize(run_nn, x_train_norm[0].reshape(989), 
    method='L-BFGS-B', jac=None, bounds= Bounds(0,1), tol=None, callback=None, 
    options={'disp': None, 'maxcor': 10, 'ftol': 2.220446049250313, 
    'gtol': 1e-05, 'eps': 0.1, 'maxfun': 500, 
    'maxiter': 500, 'iprint': - 1, 'maxls': 20, 
    'finite_diff_rel_step': None})

In [ ]:
pd.DataFrame(res.x).to_csv("./data/10000-samples/results_10000.csv", index=None)

### Visualize

In [ ]:
probabilities = pd.read_csv("./data/10000-samples/results_10000.csv")
probabilities = probabilities.to_numpy()
probabilities = probabilities.reshape((989))
# probabilities = probabilities.reshape((-1,989))

In [ ]:
scene = "./input/random-32-32-10/scen-even/random-32-32-10-even-" + str(0) + ".scen"
exp = experiment.Experiment(scene=scene)

In [ ]:
output = optVoronoiDirected.get_results(probabilities, exp)

vordir_paths, vordir_ft, vordir_u1, vordir_u2, vordir_con, vordir_conmax, vordir_conavg, vordir_G0, vordir_G, vordir_thres = output

In [ ]:
def showAccSolution(result_graph, paths, image, nodes, start_nodes, end_nodes, all_path = True, path_num = 0):
    edges_in_path = []
    image2 = 1-image
    fig, ax = plt.subplots(figsize=(12,12))
    img = np.array(1-image)
    ax = sb.heatmap(img)
    
    for index, path in enumerate(paths):
#         path = paths[path_num]
        for ite in range(len(path)-1):
            edges_in_path.append(np.array([path[ite],path[ite+1],0]))
            p1 = result_graph.nodes[path[ite]]['position']
            p2 = result_graph.nodes[path[ite+1]]['position']

            for r in (p1.x, p2.x, 1):
                for c in (p1.y, p2.y, 1):
                    if (Point(r,c) in np.array(nodes)[end_nodes]):
                        image2[int(r),int(c)] = 1
                    else:
                        image2[int(r),int(c)] += 0.2

        drawn = {}
        loop = result_graph.edges if all_path else edges_in_path

        for elem in loop:
            cur = result_graph.edges[elem[0], elem[1], 0]['probability']
            opposite = result_graph.edges[elem[1], elem[0], 0]['probability']
            unused = 1 - cur - opposite
            scaler = 0.7
            if cur > opposite:
                p1 = result_graph.nodes[elem[0]]['position']
                arr1 = np.array([p1.x,p1.y])
                p2 = result_graph.nodes[elem[1]]['position']
                arr2 = np.array([p2.x,p2.y])
                direction = (arr2-arr1) / np.linalg.norm(arr2 - arr1)
                mag = np.linalg.norm(arr2 - arr1)*scaler
                dx, dy = mag*direction
            else:
                p1 = result_graph.nodes[elem[1]]['position']
                arr1 = np.array([p1.x,p1.y])
                p2 = result_graph.nodes[elem[0]]['position']
                arr2 = np.array([p2.x,p2.y])
                direction = (arr2-arr1) / np.linalg.norm(arr2 - arr1)
                mag = np.linalg.norm(arr2 - arr1)*scaler
                dx, dy = mag*direction

            plt.arrow(p1.y, p1.x, dy, dx, head_width = 0.4, alpha=0.5, color = 'grey', linewidth=1)
    
#         p = start_nodes[index]
#         plt.scatter(nodes[p].y, nodes[p].x, color = 'red', linewidths=5)
#         p = end_nodes[index]
#         plt.scatter(nodes[p].y, nodes[p].x, color = 'lime', linewidths=5)
#         name = './results/vordir_path'+ str(path_num)+'.png'
#         plt.gca().invert_yaxis()
#         plt.savefig(name)
        
    for p in start_nodes:
        plt.scatter(nodes[p].y, nodes[p].x, color = 'red', linewidths=5)
    for p in end_nodes:
        plt.scatter(nodes[p].y, nodes[p].x, color = 'lime', linewidths=5)
        
#     name = './results/'+ str(15+2)+'.png'
    plt.gca().invert_yaxis()
#     plt.savefig(name)
    return fig

In [ ]:
showAccSolution(vordir_G0, 
                vordir_paths, 
                exp.image, 
                exp.nodes, 
                exp.start_nodes, 
                exp.end_nodes, 
                all_path = False, 
                path_num = 0) #path_num variables is unused

In [ ]:
# from scipy.optimize import minimize, rosen, rosen_der
# x0 = [1.3, 0.7, 0.8, 1.9, 1.2]
# res = minimize(rosen, x0, method='Nelder-Mead', tol=1e-6)
# res.x

In [ ]:
# import numpy as np
# from sklearn.linear_model import SGDRegressor
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# n_samples, n_features = 10, 5
# rng = np.random.RandomState(0)
# y = rng.randn(n_samples)
# X = rng.randn(n_samples, n_features)
# # Always scale the input. The most convenient way is to use a pipeline.
# reg = make_pipeline(StandardScaler(),
#                     SGDRegressor(max_iter=1000, tol=1e-3))
# reg.fit(X, y)

In [ ]:
# import numpy as np
# from sklearn.linear_model import LinearRegression
# from sklearn.compose import TransformedTargetRegressor
# tt = TransformedTargetRegressor(regressor=SGDRegressor(max_iter=1000, tol=1e-3),
#                                 func=np.log, inverse_func=np.exp)
# X = np.arange(4).reshape(-1, 1)
# y = np.exp(2 * X).ravel()
# tt.fit(X, y)

# tt.score(X, y)

# tt.regressor_.coef_